In [1]:
import pandas as pd

In [2]:
# %time df = pd.read_csv('large_dataset.csv')
# print(df.shape)
# df.head()

In [3]:
import vaex

In [4]:
# vaex_df = vaex.from_pandas(df)
# print(type(vaex_df))
# vaex_df.export_hdf5('large_data.hdf5')

download dataset [here](https://drive.google.com/file/d/1U__WYRE-TWFOtHGOYyXQHwzdw-VMrwgu/view)

In [5]:
vaex_df = vaex.open('large_data.hdf5')

In [6]:
vaex_df.head()

#,f1,f2,f3,f4,target
0,0.203936,-0.291191,-0.617251,-1.4613,10.7967
1,-1.19543,0.0692141,0.291861,1.97251,11.154
2,-2.56027,-0.827846,0.405551,-0.132588,-20.3427
3,-1.1312,0.252654,0.961175,1.23687,1.40764
4,0.858774,-0.461605,1.0955,-0.87158,-17.0431
5,0.63273,2.07437,-0.700978,1.36085,49.2754
6,-1.92771,0.155743,-0.394158,-1.3712,11.0902
7,-0.165462,-0.481113,-1.29813,1.75204,-41.1136
8,-1.31883,0.393728,-1.83311,-0.599728,1.87724
9,-0.671377,-0.172073,-0.75585,1.05745,-26.2623


In [7]:
vaex_df = vaex_df.shuffle()
df_train, df_test = vaex_df.ml.train_test_split(test_size=0.2)

C:\Users\thaku\anaconda3\envs\deep-learning\lib\site-packages\vaex\ml\__init__.py:28: UserWarning: Make sure the DataFrame is shuffled
  warnings.warn('Make sure the DataFrame is shuffled')


In [8]:
df_train.shape

(24000000, 5)

In [9]:
df_test.shape

(6000000, 5)

In [10]:
from vaex.ml.sklearn import IncrementalPredictor
from sklearn.linear_model import SGDRegressor

In [11]:
features = ['f1','f2','f3','f4']
target = 'target'

In [12]:
model = SGDRegressor()

In [13]:
vaex_model = IncrementalPredictor(features=features, target=target, model=model, batch_size=500000)
vaex_model.fit(df=df_train, progress='widget')

In [14]:
df_test = vaex_model.transform(df_test)

In [15]:
df_test.head()

#,f1,f2,f3,f4,target,prediction
0,0.619126,1.26097,0.314913,0.290428,24.0292,30.2878
1,1.16227,0.834109,-0.314575,0.378512,9.38004,19.9119
2,0.693387,1.6551,-0.543294,-0.0797774,38.472,39.6999
3,1.72537,-0.703418,1.18628,1.13521,-15.55,-17.0438
4,0.0527115,-0.12688,-1.12896,-0.594576,-11.096,-2.87949
5,-0.425523,-0.040336,0.167376,-1.43632,2.53407,-0.649487
6,-0.37032,0.353593,-1.23187,0.928499,32.1711,8.71458
7,-0.600286,-0.528439,-0.566247,1.1272,-22.6239,-12.3608
8,0.168639,-1.3583,0.572962,-2.51428,0.393485,-32.3565
9,1.26536,0.966459,-0.121084,-1.47002,12.8037,23.0884


In [16]:
from sklearn.metrics import r2_score, mean_absolute_error

In [17]:
print(r2_score(df_test['target'].values, df_test['prediction'].values))
print(mean_absolute_error(df_test['target'].values, df_test['prediction'].values))

0.5921652228866059
15.957230831174567
